In [1]:
import face_recognition
import cv2
import matplotlib.pyplot as plt
import numpy
from PIL import Image, ImageDraw

In [2]:
import time
import threading
try:
    from greenlet import getcurrent as get_ident
except ImportError:
    try:
        from thread import get_ident
    except ImportError:
        from _thread import get_ident


class CameraEvent(object):
    """An Event-like class that signals all active clients when a new frame is
    available.
    """
    def __init__(self):
        self.events = {}

    def wait(self):
        """Invoked from each client's thread to wait for the next frame."""
        ident = get_ident()
        if ident not in self.events:
            # this is a new client
            # add an entry for it in the self.events dict
            # each entry has two elements, a threading.Event() and a timestamp
            self.events[ident] = [threading.Event(), time.time()]
        return self.events[ident][0].wait()

    def set(self):
        """Invoked by the camera thread when a new frame is available."""
        now = time.time()
        remove = None
        for ident, event in self.events.items():
            if not event[0].isSet():
                # if this client's event is not set, then set it
                # also update the last set timestamp to now
                event[0].set()
                event[1] = now
            else:
                # if the client's event is already set, it means the client
                # did not process a previous frame
                # if the event stays set for more than 5 seconds, then assume
                # the client is gone and remove it
                if now - event[1] > 5:
                    remove = ident
        if remove:
            del self.events[remove]

    def clear(self):
        """Invoked from each client's thread after a frame was processed."""
        self.events[get_ident()][0].clear()


class BaseCamera(object):
    thread = None  # background thread that reads frames from camera
    frame = None  # current frame is stored here by background thread
    last_access = 0  # time of last client access to the camera
    event = CameraEvent()

    def __init__(self):
        """Start the background camera thread if it isn't running yet."""
        if BaseCamera.thread is None:
            BaseCamera.last_access = time.time()

            # start background frame thread
            BaseCamera.thread = threading.Thread(target=self._thread)
            BaseCamera.thread.start()

            # wait until frames are available
            while self.get_frame() is None:
                time.sleep(0)

    def get_frame(self):
        """Return the current camera frame."""
        BaseCamera.last_access = time.time()

        # wait for a signal from the camera thread
        BaseCamera.event.wait()
        BaseCamera.event.clear()

        return BaseCamera.frame

    @staticmethod
    def frames():
        """"Generator that returns frames from the camera."""
        raise RuntimeError('Must be implemented by subclasses.')

    @classmethod
    def _thread(cls):
        """Camera background thread."""
        print('Starting camera thread.')
        frames_iterator = cls.frames()
        for frame in frames_iterator:
            BaseCamera.frame = frame
            BaseCamera.event.set()  # send signal to clients
            time.sleep(0)

            # if there hasn't been any clients asking for frames in
            # the last 10 seconds then stop the thread
            if time.time() - BaseCamera.last_access > 10:
                frames_iterator.close()
                print('Stopping camera thread due to inactivity.')
                break
        BaseCamera.thread = None

In [13]:
class Camera_compare(BaseCamera):
    video_source = 0
    last_encoding = []
    encodings_core = {}
    encodings_few = {}

    @staticmethod
    def set_video_source(source):
        Camera_compare.video_source = source

    def add_to_core(encoding):
        print("Encoding found")
        if bool(Camera_compare.encodings_core):
            likehood_counter = 0
            merge_dict = {}
            for key, value in Camera_compare.encodings_core.items():
                if any(face_recognition.compare_faces(value, encoding, tolerance = 0.2)):
                    print("Found likeness in core, appending")
                    Camera_compare.encodings_core[key].append(encoding)
                    if likehood_counter > 0:
                        merge_dict[merge_num_0]=key
                    else:
                        merge_num_0 = key
                    likehood_counter += 1
            if likehood_counter == 0:
                print("No likeness found, creating new node")
                Camera_compare.encodings_core[len(Camera_compare.encodings_core)]=[encoding]       
            if bool(merge_dict):
                print("Similar nodes found, merging dict is: %s"%merge_dict)
                for key, value in merge_dict.items():
                    Camera_compare.encodings_core[key].extend(Camera_compare.encodings_core[value].pop())
                        
        else:
            print("core is empty, adding first node")
            Camera_compare.encodings_core[len(Camera_compare.encodings_core)]=[encoding]
                
    @staticmethod
    def frames():
        camera = cv2.VideoCapture(Camera_compare.video_source)
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')


        while True:
            # read current frame
            _, frame = camera.read()

            # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            rgb_frame = frame[:, :, ::-1]

            # Find all the faces and face enqcodings in the frame of video
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            #face_landmarks_list = face_recognition.face_landmarks(rgb_frame)
            
            # Loop through each face in this frame of video
            face_iter = 0
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                Camera_compare.add_to_core(face_encoding)
                # See if the face is a match for the known face(s)
                #if len(Camera_compare.last_encoding)!=0:
                #    compare_result = face_recognition.face_distance([Camera_compare.last_encoding], face_encoding)
                #else:
                #    compare_result = 0
                #if face_iter == 0:
                #    Camera_compare.last_encoding = face_encoding
           
                
        
                face_iter += 1

                #print("Difference %s"%compare_result)
                
                # Draw a box around the face
                #cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                #cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                #font = cv2.FONT_HERSHEY_DUPLEX
                #cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                
            #for face_landmarks in face_landmarks_list:
            #    pil_image = Image.fromarray(frame)
            #    d = ImageDraw.Draw(pil_image, 'RGBA')
            #    # Make the eyebrows into a nightmare
            #    print(face_landmarks['left_eyebrow'])
            #    d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
            #    d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
            #    frame = numpy.array(pil_image.getdata(),
            #        numpy.uint8).reshape(pil_image.size[1], pil_image.size[0], 3)
            # encode as a jpeg image and return it
            #yield cv2.imencode('.jpg', frame)[1].tobytes()
            yield frame
            

In [9]:
Camera_compare.encodings_core

{0: [array([-1.74101323e-01,  1.37764245e-01,  7.12150410e-02, -3.73719335e-02,
         -7.96454474e-02, -4.59493548e-02, -3.81805375e-02, -4.06329148e-02,
          9.58364531e-02, -2.04160586e-02,  1.77437723e-01, -7.32583404e-02,
         -2.39105314e-01,  3.41010839e-02,  4.95942086e-02,  1.11147493e-01,
         -1.79473296e-01, -1.51491642e-01, -9.84533131e-02, -7.49338195e-02,
          5.30427843e-02,  5.27619049e-02, -2.40720641e-02,  8.13887790e-02,
         -1.81173414e-01, -2.73229152e-01, -5.71168251e-02, -8.24135095e-02,
          1.32171750e-01, -1.22593910e-01, -2.39220001e-02, -6.13006167e-02,
         -1.50571823e-01, -9.60322842e-02,  7.50335753e-02,  5.86569086e-02,
         -1.04051799e-01, -1.32572412e-01,  2.43378565e-01,  5.74356467e-02,
         -1.32240862e-01, -1.69286951e-02,  3.32493410e-02,  3.48839581e-01,
          2.05615327e-01,  4.93409708e-02,  3.83851603e-02, -1.14111885e-01,
          6.70258105e-02, -2.91464418e-01,  8.00571814e-02,  1.80019289e-

In [14]:
import time
for i in range(10):
    frame = Camera_compare().get_frame()
    #cv2.imwrite('../data/out/%s.png'%i,frame)
    time.sleep(1)

Starting camera thread.
Encoding found
core is empty, adding first node
Encoding found
No likeness found, creating new node
Encoding found
No likeness found, creating new node
Encoding found
Found likeness in core, appending
Encoding found
No likeness found, creating new node
Encoding found
Found likeness in core, appending
Encoding found
Found likeness in core, appending
Encoding found
No likeness found, creating new node
Encoding found
No likeness found, creating new node
Encoding found
No likeness found, creating new node
Encoding found
No likeness found, creating new node
Encoding found
Found likeness in core, appending
Stopping camera thread due to inactivity.
